In [1]:
# Libaries and Tools

import glob
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import astropy.units as u

In [ ]:
# Key Varibles

resultsFolder = 'Results/'
saveName = '.Mag2.npz'
compName = '.Mag.npz'

In [ ]:
# Key Functions
def rollingAvg(dates,values,window):
    
    # Cut window in half
    halfWindow = window/2
    
    avgValues = np.empty(len(values)); avgValues[:] = np.nan
    stdValues = np.empty(len(values)); stdValues[:] = np.nan
    
    # Cycle Through Data
    for i in range(len(dates)):
        
        # Window
        minDate = np.max([dates[i] - halfWindow, dates[0]])
        maxDate = np.min([dates[i] + halfWindow, dates[-1]])
        
        # avg
        avgValues[i] = np.nanmean(values[(dates>minDate)&(dates<maxDate)])
        
        # std
        stdValues[i] = np.nanstd(values[(dates>minDate)&(dates<maxDate)])
        
    return avgValues,stdValues

In [ ]:
# Get Data
dataset = sorted(glob.glob(resultsFolder + '*' + saveName))

In [ ]:
# Stack data

# Open First .npz file and get list of "arrays"
data = np.load(dataset[0], allow_pickle=True)
lst  = data.files
    
# Get Data
dates   = data[lst[0]]
pxAreas = data[lst[1]]
MmAreas = data[lst[2]]
sMmean  = data[lst[3]]
usMmean = data[lst[4]]
cMass   = data[lst[5]]

# Compaison Data, first .npz file
compFile = dataset[0].split(saveName)[0] + compName
data     = np.load(compFile, allow_pickle=True)
lst      = data.files

# Extract Data
CompDates   = data[lst[0]]
CompPxAreas = data[lst[1]]
CompMmAreas = data[lst[2]]
CompSMmean  = data[lst[3]]
CompUsMmean = data[lst[4]]
CompCMass   = data[lst[5]]

# remaining cases
for i in range(1,len(dataset)):
    
    # Open .npz file and get list of "arrays"
    data = np.load(dataset[i], allow_pickle=True)
    lst  = data.files
    
    # Get Data
    newDates   = data[lst[0]]
    newPxAreas = data[lst[1]]
    newMmAreas = data[lst[2]]
    newSMmean  = data[lst[3]]
    newUsMmean = data[lst[4]]
    newCMass   = data[lst[5]]
    
    # Combine Data, Preveting Doubles
    for j in range(len(newDates)):
        if newDates[j] != dates[-1]:
            dates   = np.hstack([dates,   newDates[j:]])
            pxAreas = np.hstack([pxAreas, newPxAreas[j:]])
            MmAreas = np.hstack([MmAreas, newMmAreas[j:]])
            sMmean  = np.hstack([sMmean,  newSMmean[j:]])
            usMmean = np.hstack([usMmean, newUsMmean[j:]])
            cMass   = np.hstack([cMass,   newCMass[j:]])
            break
        
    # Compaison Data, .npz file
    compFile = dataset[i].split(saveName)[0] + compName
    data     = np.load(compFile, allow_pickle=True)
    lst      = data.files
    
    # Get Data
    newCompDates   = data[lst[0]]
    newCompPxAreas = data[lst[1]]
    newCompMmAreas = data[lst[2]]
    newCompSMmean  = data[lst[3]]
    newCompUsMmean = data[lst[4]]
    newCompCMass   = data[lst[5]]
    
    # Combine Data, Preveting Doubles
    for j in range(len(newCompDates)):
        if newDates[j] != dates[-1]:
            CompDates   = np.hstack([CompDates,   newCompDates[j:]])
            CompPxAreas = np.hstack([CompPxAreas, newCompPxAreas[j:]])
            CompMmAreas = np.hstack([CompMmAreas, newCompMmAreas[j:]])
            CompSMmean  = np.hstack([CompSMmean,  newCompSMmean[j:]])
            CompUsMmean = np.hstack([CompUsMmean, newCompUsMmean[j:]])
            CompCMass   = np.hstack([CompCMass,   newCompCMass[j:]])
            break

In [ ]:
print(np.sum(np.isnan(CompSMmean)))
print(np.sum(np.isnan(sMmean)))

In [ ]:
maxC = np.empty(len(cMass)); maxC[:] = np.nan
for i in range(len(maxC)):
    maxC[i] = np.max([np.abs(cMass[i].Tx/u.arcsec),np.abs(cMass[i].Ty/u.arcsec)])
    
maxCompC = np.empty(len(CompCMass)); maxCompC[:] = np.nan
for i in range(len(maxCompC)):
    maxCompC[i] = np.max([np.abs(CompCMass[i].Tx/u.arcsec),np.abs(CompCMass[i].Ty/u.arcsec)])

In [ ]:
# Sanity Check
print(CompCMass[0])
print(cMass[0])

In [ ]:
# Projection-corrected Mean Magnetic Field Density
plt.figure(figsize=[5,1.7],dpi=300)
plt.rcParams.update({'font.size': 6})
plt.scatter(dates[maxC<=500],abs(sMmean[maxC<=500]),0.6,label='Aligning HMI')
plt.scatter(CompDates[maxCompC<=500],abs(CompSMmean[maxCompC<=500]),0.4,marker='+',label='Aligning Segmentation')
plt.title(r'$|\bar{B}|$ of individual CHs within $\pm500^{\prime\prime}$')
plt.xlabel('Date')
plt.ylabel(r'$|\bar{B}|$ (G)')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Projection-corrected Mean Magnetic Field Density - Zoom In
plt.figure(figsize=[5,1.7],dpi=300)
plt.rcParams.update({'font.size': 6})
plt.scatter(dates[maxC<=500],abs(sMmean[maxC<=500]),0.6)
plt.scatter(CompDates[maxCompC<=500],abs(CompSMmean[maxCompC<=500]),0.4,marker='+')
plt.ylim(-0.5,15.5)
plt.title(r'$|\bar{B}|$ of individual CHs within $\pm500^{\prime\prime}$')
plt.xlabel('Date')
plt.ylabel(r'$|\bar{B}|$ (G)')
plt.grid()
plt.show()

In [ ]:
print('Aligning HMI:')
print('Mean absolute Magnetic Field Density in 500\" range:',
      np.nanmean(abs(CompSMmean[maxCompC<=500])), '±',
      np.nanstd(abs(CompSMmean[maxCompC<=500])), 'G' )

print('\nAligning Segmentation:')
print('Mean absolute Magnetic Field Density in 500\" range:',
      np.nanmean(abs(sMmean[maxC<=500])), '±',
      np.nanstd(abs(sMmean[maxC<=500])), 'G' )

In [ ]:
# Rolling Avg
plt.figure(figsize=[5,1.7],dpi=300)
plt.rcParams.update({'font.size': 6})
plt.scatter(dates,MmAreas,0.6)
plt.scatter(CompDates,CompMmAreas,0.4,marker='+')
plt.title('Area in Mm of Individual CHs')
plt.xlabel('Date')
plt.ylabel(r'CH Area (Mm$^2$)')
plt.grid()
plt.show()

In [ ]:
print('Aligning HMI:')
print('Mean area of all CHs:',
      np.nanmean(CompMmAreas), '±',
      np.nanstd(CompMmAreas), 'Mm2' )
print('Median Area of all CHs:',
      np.nanmedian(CompMmAreas), 'Mm2')

print('\nAligning Segmentation:')
print('Mean area of all CHs:',
      np.nanmean(MmAreas), '±',
      np.nanstd(MmAreas), 'Mm2' )
print('Median Area of all CHs:',
      np.nanmedian(MmAreas), 'Mm2')

In [ ]:
# Number of CHs
dts,Counts = np.unique(dates,return_counts=True)
compDts,compCounts = np.unique(CompDates,return_counts=True)

plt.figure(figsize=[5,5],dpi=300)
plt.rcParams.update({'font.size': 6})
plt.bar(compDts,compCounts,color=[0,0,1],label='Align HMI')
plt.bar(dts,Counts,color=[1,0,0,.5],label='Align Segmentation')
plt.title('Number of individual CHs')
plt.xlabel('Observation, organized by Date')
plt.ylabel('Number of CHs')
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=[5,1.7],dpi=300)
plt.rcParams.update({'font.size': 6})
plt.scatter(dates[maxC<=500],MmAreas[maxC<=500],0.6)
plt.scatter(CompDates[maxCompC<=500],CompMmAreas[maxCompC<=500],0.4,marker='+')
plt.title(r'Area of Individual CHs within $\pm500^{\prime\prime}$')
plt.xlabel('Date')
plt.ylabel(r'CH Area (Mm$^2$)')
plt.grid()
plt.show()